> **Jupyter slideshow:** This notebook can be displayed as slides. To view it as a slideshow in your browser, type the following in the console:


> `> jupyter nbconvert [this_notebook.ipynb] --to slides --post serve`


> To toggle off the slideshow cell formatting, click the `CellToolbar` button, then `View --> Cell Toolbar --> None`.

In [1]:
import numpy as np
from __future__ import division
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Naive Bayes

_Authors: Dan Wilhelm (LA), Alex Combs (NYC) _

---

<a id="learning-objectives"></a>
### Learning Objectives
*By the end of this lesson, you will be able to:*
- Describe Naive Bayes.
- Choose a Naive Bayes implementation based on your use case.
- Implement a Naive Bayes model through scikit-learn.

### Lesson Guide

- [Discriminative Models vs. Generative Models](#discriminative-models-vs-generative-models)
    - [Example: Discriminative vs. Generative](#discriminative-vs-generative-example)
    - [Making a Generative Model (Joint Probability Distribution)](#making-a-generative-model-joint-probability-distribution)
    - [The Joint Probability Generative Model is Not Practical](#the-joint-probability-generative-model-is-not-practical)
    - [Making a Discriminative Model (Logistic Regression)](#making-a-discriminative-model-logistic-regression)


- [A Better Generative Model?](#a-better-generative-model)
    - [Bayes' Theorem](#bayes-theorem)
    - [Conditional Probability](#conditional-probability)


- [A Simple Spam Example](#a-really-simple-spam-example)
    - [Problem: Multiple Features Require Joint Probabilities](#problem-multiple-features-require-joint-probabilities)
    - [Solution: The Naive Bayes Independence Assumption](#solution-the-naive-bayes-independence-assumption)
    - [Spam Application: Multiple Features](#spam-application-multiple-features)


- [Production Issues](#production-issues)
- [Summary](#summary)
- [Implementation in Scikit-Learn](#implementation-in-scikit-learn)
- [Guided Practice: Scikit-Learn Implementation](#guided-practice-scikit-learn-implementation)
- [Conclusion](#conclusion)


<a id="discriminative-models-vs-generative-models"></a>
## Discriminative Models vs. Generative Models
---

Logistic regression is a **discriminative model**. 

+ It's equation, $P\;(\;class\;|\;features\;)$, **discriminates** between two classes. In other words, it describes the boundary between the classes.
+ From this equation, we can’t generate "typical" members of either class — we only know their boundaries.

Naive Bayes is a **generative model**. It can **generate** typical members of each class, as we know what a typical member of each class looks like.

+ Note: We are still estimating $P\;(\;class\;|\;features\;)$ for each class.
+ By computing how typical each example is to each class, we can choose the most likely class.

<a id="discriminative-vs-generative-example"></a>
### Example: Discriminative vs. Generative

Let's build a simple generative model from scratch. Suppose we are attempting to infer whether or not someone is a GA data science student. To do this, we take members of the general population and evaluate three binary features:

- **G**: At GA.
- **C**: Has computer.
- **S**: Has stats book.

Our model of a "person" is now the presence or absence of each of these three features.

#### Data

We sample 10 students in each class. We'll assume that the GA/non-GA numbers are good, representative samples of the population for each combination of features. 

- **GA student**: GCS, GC, GCS, GCS, GC, C, C, GC, GCS, GCS.
- **Non-GA student**: none, none, C, GC, C, none, C, C, CS, none.

We grabbed 10 examples per class. This is typically the data we have access to in most data science problems — examples of each class of interest. However, to be representative of the overall population, should we have gotten 10 examples from each of the eight categories instead? Which technique would be more accurate? Think about it while we take this toy data and tally.

<a id="making-a-generative-model-joint-probability-distribution"></a>
### Making a Generative Model (Joint Probability Distribution)

Summarize this data per class (where $\neg$ indicates negation):

_                 | GCS | GC | CS | S | C | None   
---               | --- | -- | -- | - | - | -    
**Student**       | 5   | 3  | 0  | 0 | 2 | 0     
**$\neg$Student** | 0   | 1  | 1  | 0 | 4 | 4     


From these, we'll make a generative model of a student:

- $P(\;Student\;|\;GCS\;) = 5/5 = 1$.
- $P(\;Student\;|\;GC\;) = 3/(3+1) = 3/4$.
- $P(\;Student\;|\;CS\;) = 0$.
- $P(\;Student\;|\;S\;) = NA$.
- $P(\;Student\;|\;C\;) = 2/(4+2) = 1/3$.
- $P(\;Student\;|\;none\;) = 0$.

We'll also make a model for a non-student:

- $P(\;\neg Student\;|\;GCS\;) = 0/5 = 0$.
- $P(\;\neg Student\;|\;GC\;) = 1/4$.
- $P(\;\neg Student\;|\;CS\;) = 1$.
- $P(\;\neg Student\;|\;S\;) = NA$.  
- $P(\;\neg Student\;|\;C\;) = 4/(4+2) = 2/3$.
- $P(\;\neg Student\;|\;none\;) = 4/4 = 1$.


You may recognize that we just computed the **joint probabilities**. 

Note: We must store $2^3$ parameters in total — the presence or absence of three features.

#### Using the Generative Model

+ Suppose we see someone with $GCS$. We would then guess with a confidence of one that they are a GA student. If someone is $GC$, we would guess that they are a GA student with a confidence of $3/4$.

+ Suppose we want to generate a sequence of typical GA students. It's easy! With probability $\frac{1}{1 + 3/4 + 1/3} = \frac{12}{25}$, generate a $GCS$ person. With probability $\frac{3/4}{1 + 3/4 + 1/3} = \frac{9}{25}$, generate a $GC$ person.

<a id="the-joint-probability-generative-model-is-not-practical"></a>
### The Joint Probability Generative Model is Not Practical

Earlier, we saw how to build a generative model strictly from joint probabilities. However, this method has some major problems.

+ The number of parameters stored in the model increases exponentially. For example, if each feature is binary and we have 100 features, we would need $2^{100} \approx 10^{30}$ parameters for every joint probability.

+ We would need _enormous_ amounts of data to ensure we have sufficient training examples to evaluate each joint probability robustly (again, just to emphasize — $2^{100}$ joint probabilities for only $100$ binary features).

Although each probability is easy to calculate, the joint probability model is simply not practical.

<a id="making-a-discriminative-model-logistic-regression"></a>
### Making a Discriminative Model (Logistic Regression)

For comparison, let's make a discriminative model using the same data. 

| _                  | GCS | GC | CS | S | C | None
| ---               | --- | -- | -- | - | - | -
| **Student**       | 5   | 3  | 0  | 0 | 2 | 0 
| **$\neg$Student** | 0   | 1  | 1  | 0 | 4 | 4

We'll eyeball a hyperplane that separates the classes. For simplicity, let's suppose that our hyperplane (and corresponding link function) is the following formula, where $\sigma$ is the logistic function and $X$ is our feature vector. $G = 1$ if true and $G = -1$ if false:

$$P(\;Student\;|\;X\;) = \sigma(5G - 2C + S)$$

We can see that this model is approximately correct. Recall that $5G - 2C + S > 0$ indicates a probability greater than 0.5. This model makes sense — being seen at GA is a strong indicator of being a GA student. However, having a computer is a less positive signal, as many non-students also have computers.

### Using the Discriminative Model

+ We can predict the probability that $GC$ is a student by letting $G = 1, C = 1, S = -1$. Because $5G - 2C + S = 2 > 0$, they're likely a student.

+ This model is more compact, as we store fewer parameters — $4$ (three plus a bias) instead of $2^3$. (It even scales linearly instead of exponentially.)

+ However, we can't generate typical students with any accuracy. (In this case we can attempt to, but with any substantial number of features, typical members would be far too ambiguous.)

<a id="a-better-generative-model"></a>
## A Better Generative Model?
---

There are some reasonable simplifications we can make to build a practical generative model, and they're nearly as simple to calculate.

Let's understand why we're doing this. Recall that we’re looking for this: $P\;(\;class\;|\;features\;)$. Here, class refers to a category such as "vertosa," "versicola," "Student," etc.

+ Computing this directly implies that we have a lot of data for every feature combination, which isn't typically the case. Usually, our training data only ensure we have sufficient examples for each **class**. For example, 10 examples of GA students and 10 examples of non-GA students.

+ It would be great if we could flip the conditional probability, as our training data have a lot of examples of each class. The Bayes' Theorem will help us here.

<a id="bayes-theorem"></a>
### Bayes' Theorem

### $$P\left(\;class\;|\;features\;\right) = \frac{P\left(\;features\;|\;class\;\right)P\left(\;class\;\right)}{P(\;features\;)} $$
 
Luckily, it's easy to compute these probabilities directly from a data table. 

+ **$P(\;class\;)$**. For example: $P(\;student\;) = P(\;\neg student\;) = 1/2$.
+ **$P(\;features\;)$**. For example, we had five $GCS$ combinations in 20 examples, so $P(\;GCS\;) = 5/20 = 1/4$.
+ **$P(\;features\;|\;class\;)$**. For example, for $GCS$ we had five students of 10. $P(\;GCS\;|\;Student\;) = 1/2$.

| _                 | GCS | GC | CS | S | C | None
| ---               | --- | -- | -- | - | - | -
| **Student**       | 5   | 3  | 0  | 0 | 2 | 0 
| **$\neg$Student** | 0   | 1  | 1  | 0 | 4 | 4


Then, what is $P(\;Student\;|\;GCS\;)$? It's just $\frac{1/2 * 1/2}{1/4} = 1$ — exactly what we computed above as a joint probability.

+ Does this work out for all of our earlier joint probabilities? See if you can understand why.
+ Hint: Recall that Bayes' Theorem is just a single conditional probability. The numerator is just the chain rule: $P(\;A \cap B\;) = P(\;A\;|\;B\;)\;P(\;B\;)$.

| _                  | GCS                           | GC                 | ... | 
| ---                | ---------------------------   | ------------------ | --  | 
| **Student**        | Student $\cap$ GCS            | Student $\cap$ GC  | ... | P(Student) = 1/2
| **$\neg$ Student** | $\neg$ Student $\cap$ GCS     | $\neg$ Student $\cap$ GC | ... | P($\neg$ Student) = 1/2
| probability         | P(GCS) = 5/20                 | P(GC) = 4/20       |   ...  | 

<a id="conditional-probability"></a>
### Conditional Probability

Let's review and see how the Bayes' Theorem is just a big conditional probability.

In general, the **conditional probability** for events $A$ and $B$ is:

### $$ P(\;A\;|\;B\;) = \frac{P(\;A \cap B\;)}{P(\;B\;)} $$

Hence (because $\cap$ is commutative — i.e., $P(A \cap B) = P(B \cap A)$):

### $$ P(A \cap B) = P(A\;|\;B) \; P(B) = P(B\;|\;A) \; P(A) $$

This is often referred to as the "chain rule" of probability.

### Bayes' Theorem

Substitute the second equation above into the first:

### $$P\left(\;A\;|\;B\;\right) = \frac{P(\;A \cap B\;)}{P(\;B\;)} = \frac{P\left(\;B\;|\;A\;\right)P\left(\;A\;\right)}{P(\;B\;)}$$


### $$P\left(\;class\;|\;features\;\right) = \frac{P\left(\;features\;|\;class\;\right)P\left(\;class\;\right)}{P(\;features\;)} $$

As we saw earlier, it's very easy to compute $P(\;features\;|\;class\;)$ from the training data.

<a id="a-really-simple-spam-example"></a>
## A Simple Spam Example
---

Here's another example. We're trying to predict spam emails. For now, we have one feature: whether or not the email mentions "guarantee."

$G$ = Guarantee, $S$ = Is spam.

 $$P\left(\;S\;|\;G\;\right) = \frac{P\left(\;G\;|\;S\;\right)P\left(\;S\;\right)}{P(\;G\;)} = \frac{P\left(\;G\;|\;S\;\right)P\left(\;S\;\right)}{P(\;G\;|\;S)P(\;S\;) + P(\;G\;|\;\neg{S})P(\;\neg{S}\;)}$$

Earlier, we saw how it's possible to compute $P\;(\;G\;)$ directly. In many books, you'll see it computed in this alternative fashion, which is based on our classes rather than on every combination of features. 

> The denominator looks complicated, but it actually isn't. Because $G$ is binary (either present or not), then:

> $$P(G) = P(G \cap S) + P(G \cap \neg{S})$$

> Now, simply expand each term using the chain rule and you’ll get the denominator.

Remember that we started with a term $P\;(\;features\;)$ — in general, using this would require the calculation of every combination of features (i.e., we would also need to compute $P\;(\;GS\;)$, $P\;(\;GCS\;)$, etc.). However, by expanding $P\;(\;G\;)$ via the chain rule, we got an expression that depends only on the individual classes about which we have data.

<a id="problem-multiple-features-require-joint-probabilities"></a>
### Problem: Multiple Features Require Joint Probabilities

In this spam example, we only had one feature: $G$. But, in general, we'll probably use more than one. Really, we want to see some feature vector $X_1, X_2, ..., X_n$:

$$P\left(\;S\;|\;X_1, ..., X_n\;\right) = \frac{P\left(\;X_1,  ..., X_n\;|\;S\;\right)}{P(\;X_1,  ..., X_n\;|\;S) + P(\;X_1, ..., X_n\;|\;\neg{S})}$$

For example, what is the likelihood that something is spam given that the email mentions "guarantee," "oil," "prince," and "Nigeria," but not "meeting," "colleague," and "dad?"

With a lot of features, calculating the joint probabilities quickly becomes complicated. We would definitely need a lot of data to ensure we have enough feature combinations. If you reason this out, you may quickly realize that we run into the same joint probability problem as before, requiring exponentially more joint probabilities.

No matter how diligent we are, we may never collect a single training example that contains the precise combination of feature words we need. Therefore, we would be unable to classify a new email containing a particular combination of words.

<a id="solution-the-naive-bayes-independence-assumption"></a>
### Solution: The Naive Bayes Independence Assumption

We're stuck, because conditional joint probabilities are required for multiple features. This means there are exponentially more probabilities to compute and data to collect.

To get around this, let's make an assumption: **All $X_i$ are conditionally independent given $S$** (where $S$ indicates "is spam"). This means that, given $S$, no two $X_i$ depend on each other. For example, the words "Nigeria" and "prince" each independently indicate that an email is spam. So, it's not the complex interaction of words that determines spam; each feature can indicate whether or not an email is spam independently.

> Of course, this assumption is rarely (if ever) true. Often, it requires precise reading to tell whether or not an email was written by a native speaker. In this case, it's often not the particular words used, but how they are used in context.

Recall that if events $A$ and $B$ are independent, then the probability $P\;(\;AB\;) = P\;(\;A\;)\;P\;(\;B\;)$. Similarly, if $A$ and $B$ are conditionally independent on $S$, then $P\;(\;AB\;|\;S\;) = P\;(\;A\;|\;S\;)\;P\;(\;B\;|\;S\;)$.

> This formula works out well in general, too:

> $$P\left(\;X_{1}X_{2} \dots X_{n}\;|\;S\;\right) = P\left(\;X_{1} |\;S\;\right) * P\left(\;X_{2} |\;S\;\right) ... P\left(\;X_{n} |\;S\;\right)$$

To see if this **conditional independence assumption** might simplify the numerator $P(X_1, ..., X_n\;|\;S)$ by removing the joint probabilities, let's apply the definition of conditional probability, followed by the application of the independence assumption:

$$P\;(\;SGM\;) = P\;(\;GM\;|\;S\;)\;P\;(\;S\;) = P\;(\;G\;|\;S\;)\;P\;(\;M\;|\;S\;)\;P\;(\;S\;)$$


> None of these probabilities require us to examine multiple features at once in our data set, making them drastically easier to compute. For example, $P(\;A\;|\;S\;)$ could indicate only the probability of the word “a” occurring in a spam email.

In reality, model parameters/coefficients are unlikely to be independent. But Naive Bayes makes exactly this assumption and often works well despite this.

<a id="spam-application-multiple-features"></a>
### Spam Application: Multiple Features

How is this used in practice? Let's combine the Naive Bayes simplification above with our original formula (the denominator is computed the same way as before, combined with our naive assumption):

 $$P\left(\;S\;|\;GM\;\right) = \frac{P(\;SGM\;)}{P(\;GM\;)} = \frac{P\left(\;GM\;|\;S\;\right)P\left(\;S\;\right)}{P(\;GM\;|\;S)P(\;S\;) + P(\;GM\;|\;\neg{S})P(\;\neg{S}\;)} = \frac{P\left(\;G\;|\;S\;\right)P\left(\;M\;|\;S\;\right)P\left(\;S\;\right)}{P(\;G\;|\;S)P(\;M\;|\;S)P(\;S\;) + P(\;G\;|\;\neg{S})P(\;M\;|\;\neg{S})P(\;\neg{S}\;)}$$
 
Typically, we compute this probability for each class (in this case, just $S$ or not $S$), then predict the class with the highest probability. Note that, for all of these, the denominator $P(GM)$ is constant. Hence, this formula is often written as "proportional" ($\propto$), considerably simplifying it. Instead of comparing the exact probabilities, we can simply see how they score relative to each other. So:

 $$P\left(\;S\;|\;GM\;\right) \propto P\left(\;G\;|\;S\;\right)P\left(\;M\;|\;S\;\right)P\left(\;S\;\right)$$

Remember, don't be scared by the denominator!

<a id="production-issues"></a>
## Production Issues
---

Recall that Naive Bayes is proportional to:

 $$P\left(\;S\;|\;GM\;\right) \propto P\left(\;G\;|\;S\;\right)P\left(\;M\;|\;S\;\right)P\left(\;S\;\right)$$

Accidentally using a zero probability for any of these could present major problems — the entire probability estimation would be zero.

- **New features:** What if a particular feature was never seen in our training data? Instead of using a zero probability, we should use a technique such as [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) to estimate a small non-zero probability.

- **Underflow:** Probabilities could be very small if some features rarely occur in some classes. Recall that floating point often gives us trouble because of its limited precision — small floats tend toward zero. We can approach this problem by storing the logarithm of each probability, $P_i$, instead of $P_i$ itself:

$$log(P_1P_2) = log\ P_1 + log\ P_2$$

$$e^{log\ P_1} = P_1$$

So: $$P_1P_2 \dots P_n = e^{log\ P_1 + ... + log\ P_n}$$.

<a id="summary"></a>
## Summary
---

**Why is the Naive Bayes formula important?** With the independence assumption, we don't need to compute every joint probability distribution. Even if none of the training data contain the words “guarantee” and “millions,” we only need to compute the probability of each word separately: $P(\;G\;|\;S\;)$ and $P(\;M\;|\;S\;)$. 

These calculations can be performed quickly using our training data. The downside is that, if spam is actually determined by some complex interaction between "guarantee" and "millions" (e.g., only the presence of one but not the other), then the independence assumption does not hold and our model will not have the capacity to predict spam correctly.

To make Naive Bayes a classifier, all we have to do is compute the probability of $P(y|X)$ for each class, $y$. In math notation, this is:

### $$ P(y \;|\; x_1, ..., x_n) \propto P(y) \prod_{i=1}^n P(x_i \;|\; y) \\
\downarrow \\
\hat{y} = arg \; \underset{y}{max} \; P(y) \prod_{i=1}^n P(x_i \;|\; y)$$

> Recall that $arg\underset{y}{max}$ means we find the class in the vector of categories, $y$, that gives us the maximum expression value.

<a id="implementation-in-scikit-learn"></a>
## Implementation in Scikit-Learn
---


- [Docs 1](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
- [Docs 2](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)
- [Docs 3](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html)

<img src="./images/naive-bayes.png">

The differences can be summarized as:
-    ***BernoulliNB*** is designed for binary/Boolean features.
-    The ***multinomial Naive Bayes classifier*** is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as `tf-idf` may also work.
-    ***GaussianNB*** is designed for continuous features (that can be scaled between zero and one) and is assumed to be normally distributed.

<a id="guided-practice-scikit-learn-implementation"></a>
## Guided Practice: Scikit-Learn Implementation

In [9]:
from sklearn import naive_bayes
import numpy as np
import pandas as pd

data = pd.read_csv('./datasets/spam_base.csv')

In [10]:
data.head()

,0,0.64,0.64.1,0.1,0.32,0.2,0.3,0.4,0.5,0.6,...,0.40,0.41,0.42,0.778,0.43,0.44,3.756,61,278,1
0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
2,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.00,0.223,0.0,0.000,0.000,0.000,3.000,15,54,1


> **Check**: What do you think is going on with this data set?

> Which scikit-learn Naive Bayes implementation should we use?

In [11]:
feature_set = data.iloc[:, :-1]
target = data.iloc[:, -1]

classifier1 = naive_bayes.MultinomialNB().fit(feature_set, target)

In [12]:
from sklearn.model_selection import cross_val_score
cross_val_score(classifier1, feature_set, target, cv=5)

array([0.79153094, 0.81976113, 0.81413043, 0.7867247 , 0.69749728])

> **Check**: Is that good?

In [13]:
1. - np.mean(target)

0.606086956521739

<a id="conclusion"></a>
## Conclusion
---


How does Naive Bayes fit into your toolkit? What are its pros and cons? How do you choose between variants?

#### Additional Resources

- [An interesting slide](https://web.stanford.edu/class/cs124/lec/naivebayes.pdf) from a Stanford MOOC that has a section on Naive Bayes.
- [A much more technical paper](https://www.cs.cmu.edu/~tom/mlbook/NBayesLogReg.pdf) comparing Naive Bayes to logistics regressions.
- [More exposition on Naive Bayes](http://blog.yhat.com/posts/naive-bayes-in-python.html).
- [Naive Bayes from scratch](http://machinelearningmastery.com/naive-bayes-classifier-scratch-python/).